In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('2008.csv')

In [3]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,CancellationCode,6872294,1.960618
1,LateAircraftDelay,5484993,21.751700
2,NASDelay,5484993,21.751700
3,WeatherDelay,5484993,21.751700
4,CarrierDelay,5484993,21.751700
5,SecurityDelay,5484993,21.751700
6,AirTime,154699,97.793081
7,ActualElapsedTime,154699,97.793081
8,ArrDelay,154699,97.793081
9,ArrTime,151649,97.836592


In [3]:
df = df.drop(['CancellationCode', 'LateAircraftDelay', 'NASDelay', 'WeatherDelay', 'CarrierDelay', 'SecurityDelay',
             'ArrTime', 'ActualElapsedTime', 'Cancelled', 'Year', 'TaxiIn',
             'TaxiOut', 'Diverted', 'Cancelled', 'DepDelay', 'DepTime'], axis=1)
df.head()

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance
0,1,3,4,1955,2225,WN,335,N712SW,150.0,116.0,-14.0,IAD,TPA,810
1,1,3,4,735,1000,WN,3231,N772SW,145.0,113.0,2.0,IAD,TPA,810
2,1,3,4,620,750,WN,448,N428WN,90.0,76.0,14.0,IND,BWI,515
3,1,3,4,930,1100,WN,1746,N612SW,90.0,78.0,-6.0,IND,BWI,515
4,1,3,4,1755,1925,WN,3920,N464WN,90.0,77.0,34.0,IND,BWI,515


In [4]:
df = df.dropna()
df.head()

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,CRSElapsedTime,AirTime,ArrDelay,Origin,Dest,Distance
0,1,3,4,1955,2225,WN,335,N712SW,150.0,116.0,-14.0,IAD,TPA,810
1,1,3,4,735,1000,WN,3231,N772SW,145.0,113.0,2.0,IAD,TPA,810
2,1,3,4,620,750,WN,448,N428WN,90.0,76.0,14.0,IND,BWI,515
3,1,3,4,930,1100,WN,1746,N612SW,90.0,78.0,-6.0,IND,BWI,515
4,1,3,4,1755,1925,WN,3920,N464WN,90.0,77.0,34.0,IND,BWI,515


In [5]:
df['Late'] = np.where(df['ArrDelay'] >=30, 1, 0)
df = df.drop(['ArrDelay'], axis = 1)
df.describe()

,Month,DayofMonth,DayOfWeek,CRSDepTime,CRSArrTime,FlightNum,CRSElapsedTime,AirTime,Distance,Late
count,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06,6.855024e+06
mean,6.388525e+00,1.573826e+01,3.925802e+00,1.325037e+03,1.493715e+03,2.212505e+03,1.290778e+02,1.040186e+02,7.287441e+02,1.352895e-01
std,3.398833e+00,8.801085e+00,1.988459e+00,4.643130e+02,4.826643e+02,1.956435e+03,6.953273e+01,6.743981e+01,5.632449e+02,3.420326e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,-1.410000e+02,0.000000e+00,1.100000e+01,0.000000e+00
25%,3.000000e+00,8.000000e+00,2.000000e+00,9.250000e+02,1.115000e+03,6.180000e+02,8.000000e+01,5.500000e+01,3.260000e+02,0.000000e+00
50%,6.000000e+00,1.600000e+01,4.000000e+00,1.317000e+03,1.515000e+03,1.563000e+03,1.110000e+02,8.600000e+01,5.810000e+02,0.000000e+00
75%,9.000000e+00,2.300000e+01,6.000000e+00,1.715000e+03,1.905000e+03,3.494000e+03,1.590000e+02,1.320000e+02,9.540000e+02,0.000000e+00
max,1.200000e+01,3.100000e+01,7.000000e+00,2.359000e+03,2.359000e+03,9.741000e+03,6.600000e+02,1.350000e+03,4.962000e+03,1.000000e+00


In [7]:
df.shape

(6855024, 14)

In [6]:
from sklearn.preprocessing import LabelEncoder

le_UniqueCarrier = LabelEncoder()
le_Origin = LabelEncoder()
le_Dest = LabelEncoder()

df['UniqueCarrier_n'] = le_UniqueCarrier.fit_transform(df['UniqueCarrier'])
df['Origin_n'] = le_Origin.fit_transform(df['Origin'])
df['Dest_n'] = le_Dest.fit_transform(df['Dest'])

df = df.drop(['UniqueCarrier', 'Origin', 'Dest', 'TailNum'], axis=1)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6855024 entries, 0 to 7009727
Data columns (total 13 columns):
Month              int64
DayofMonth         int64
DayOfWeek          int64
CRSDepTime         int64
CRSArrTime         int64
FlightNum          int64
CRSElapsedTime     float64
AirTime            float64
Distance           int64
Late               int64
UniqueCarrier_n    int64
Origin_n           int64
Dest_n             int64
dtypes: float64(2), int64(11)
memory usage: 732.2 MB


In [7]:
X = df.drop('Late', 1)
y = df['Late']


In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=10).fit_transform(X, y)
X_new.shape

(6855024, 10)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.999, random_state=0)


In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()

cross_val_score(rfc, X_train, y_train, cv=10)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 

array([0.84693878, 0.85131195, 0.84693878, 0.85714286, 0.85568513,
       0.85276968, 0.85276968, 0.85985401, 0.85233918, 0.85380117])